In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../.env", override=True)

True

In [ ]:
from anthropic import Anthropic

client = Anthropic(
    base_url=os.environ["ANTHROPIC_BASE_URL"],
    auth_token=os.environ["ANTHROPIC_AUTH_TOKEN"],
)

In [3]:
response = client.messages.create(
    model="us.anthropic.claude-opus-4-1-20250805-v1:0",
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello, world"}],
)
print(response)


Message(id='msg_bdrk_01EQEjz3RxRJ6CSJNvhVynAx', content=[TextBlock(citations=None, text="Hello! Welcome to our conversation. How are you doing today? Is there anything on your mind that you'd like to talk about or any questions I can help you with?", type='text')], model='claude-opus-4-1-20250805', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=None, cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=10, output_tokens=38, server_tool_use=None, service_tier=None))


In [7]:
with client.messages.stream(
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello"}],
    model="us.anthropic.claude-opus-4-1-20250805-v1:0",
) as stream:
    for s in stream:
        print(s, flush=True)

RawMessageStartEvent(message=Message(id='msg_bdrk_01Mt1X8UshcjzkPrac6fEVFv', content=[], model='claude-opus-4-1-20250805', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=Usage(cache_creation=None, cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=8, output_tokens=3, server_tool_use=None, service_tier=None)), type='message_start')
RawContentBlockStartEvent(content_block=TextBlock(citations=None, text='', type='text'), index=0, type='content_block_start')
RawContentBlockDeltaEvent(delta=TextDelta(text='Hello! How', type='text_delta'), index=0, type='content_block_delta')
TextEvent(type='text', text='Hello! How', snapshot='Hello! How')
RawContentBlockDeltaEvent(delta=TextDelta(text=' can I help you', type='text_delta'), index=0, type='content_block_delta')
TextEvent(type='text', text=' can I help you', snapshot='Hello! How can I help you')
RawContentBlockDeltaEvent(delta=TextDelta(text=' today?', type='text_delta'), index=0, type='conten

In [5]:
with client.messages.stream(
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello"}],
    model="us.anthropic.claude-opus-4-1-20250805-v1:0",
) as stream:
    for s in stream.text_stream:
        print(s, end="", flush=True)


Hello! How can I help you today?

In [8]:
with client.messages.stream(
    model="us.anthropic.claude-opus-4-1-20250805-v1:0",
    max_tokens=16000,
    thinking={"type": "enabled", "budget_tokens": 10000},
    messages=[{"role": "user", "content": "What is 27 * 453?"}],
) as stream:
    thinking_started = False
    response_started = False

    for event in stream:
        if event.type == "content_block_start":
            print(f"\nStarting {event.content_block.type} block...")
            # Reset flags for each new block
            thinking_started = False
            response_started = False
        elif event.type == "content_block_delta":
            if event.delta.type == "thinking_delta":
                if not thinking_started:
                    print("Thinking: ", end="", flush=True)
                    thinking_started = True
                print(event.delta.thinking, end="", flush=True)
            elif event.delta.type == "text_delta":
                if not response_started:
                    print("Response: ", end="", flush=True)
                    response_started = True
                print(event.delta.text, end="", flush=True)
        elif event.type == "content_block_stop":
            print("\nBlock complete.")



Starting thinking block...
Thinking: I need to calculate 27 * 453.

Let me break this down:
27 * 453
= 27 * (400 + 50 + 3)
= 27 * 400 + 27 * 50 + 27 * 3
= 10800 + 1350 + 81
= 12231

Let me double-check this:
27 * 453
= 453 * 27
= 453 * (20 + 7)
= 453 * 20 + 453 * 7
= 9060 + 3171
= 12231

Both methods give me 12231.
Block complete.

Starting text block...
Response: 27 * 453 = 12,231
Block complete.


In [10]:
response = client.messages.create(
    model="us.anthropic.claude-opus-4-1-20250805-v1:0",
    max_tokens=1024,
    tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        }
    ],
    messages=[{"role": "user", "content": "What's the weather like in San Francisco?"}],
)
print(response)

Message(id='msg_bdrk_01M67MYmTwosWQ42RkYQVNj7', content=[ToolUseBlock(id='toolu_bdrk_01G5i7AdQ55WppfGbE4TVHkZ', input={'location': 'San Francisco, CA'}, name='get_weather', type='tool_use')], model='claude-opus-4-1-20250805', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation=None, cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=403, output_tokens=56, server_tool_use=None, service_tier=None))
